In [57]:
from sympy import symbols, integrate, Matrix, solve, sin, pi, lambdify, pprint
import numpy as np
import matplotlib.pyplot as plt

# Define constants
E = 200e9  # Pa
nu = 0.3
h = 0.2  # m
q0 = 4e3  # N/m^2
a = 8  # m
b = 8  # m
D = E * h**3 / (12 * (1 - nu**2))  # Flexural rigidity ≈ 1.4652e8 N·m

# Define symbolic variables
x, y = symbols('x y')

# Basis functions up to degree 6
basis_indices = [(1,1), (2,1), (1,2), (2,2), (3,1), (1,3)]  # 6 terms
phi = [x**(i-1) * (a-x) * y**(j-1) * (b-y) for i, j in basis_indices]

In [45]:
# Compute K_ij and F_i
def compute_system(n):
    K = Matrix.zeros(n, n)
    F = Matrix.zeros(n, 1)
    for i in range(n):
        for j in range(n):
            w_xx = phi[i].diff(x, 2)
            w_yy = phi[i].diff(y, 2)
            v_xx = phi[j].diff(x, 2)
            v_yy = phi[j].diff(y, 2)
            w_xy = phi[i].diff(x, y)
            v_xy = phi[j].diff(x, y)
            integrand = D * (w_xx * v_xx + w_yy * v_yy + nu * (w_xx * v_yy + w_yy * v_xx) + 2 * (1 - nu) * w_xy * v_xy)
            K[i, j] = integrate(integrand, (x, 0, a), (y, 0, b))
        F[i] = integrate(q0 * phi[i], (x, 0, a), (y, 0, b))
    return K, F

In [46]:
# Compute potential energy
def compute_potential_energy(K, F, solution, n):
    c = Matrix([solution.get(symbols(f'c{i+1}'), 0) for i in range(n)])
    return (0.5 * c.T * K * c - c.T * F)[0]

In [47]:
# Compute Ritz approximations and potential energy
solutions = {}
potential_energies = []
for n in range(1, 7):
    K, F = compute_system(n)
    c = Matrix(symbols(f'c1:{n+1}'))
    solution = solve(K * c - F, c)
    w_approx = sum(solution.get(symbols(f'c{i+1}'), 0) * phi[i] for i in range(n))
    pi_energy = compute_potential_energy(K, F, solution, n)
    solutions[n] = (K, F, solution, w_approx, pi_energy)
    potential_energies.append(float(pi_energy.subs(a, 8).subs(b, 8).evalf()))

In [48]:
# Analytical solution (Navier’s method)
m, n = symbols('m n', integer=True)
w_mn = (16 * q0 / (pi**2 * m * n)) / (D * pi**4 * ((m**2 / a**2) + (n**2 / b**2))**2) * sin(m * pi * x / a) * sin(n * pi * y / b)
w_analytical = 0
m_max, n_max = 5, 5
for i in range(1, m_max + 1, 2):
    for j in range(1, n_max + 1, 2):
        w_analytical += w_mn.subs({m: i, n: j})
w_analytical_func = lambdify((x, y), w_analytical.subs({a: 8, b: 8, D: D, q0: 5e3}), 'numpy')


In [49]:
# Numerical evaluation along y = b/2
x_vals = np.linspace(0, 8, 100)
y_val = 4
w_analytical_vals = w_analytical_func(x_vals, y_val)


In [50]:
print(K.shape)

(6, 6)


In [51]:
print(F.shape)

(6, 1)


In [52]:
# Plot displacement
plt.figure(figsize=(10, 6))
plt.plot(x_vals, w_analytical_vals, 'k-', label='Analytical Solution', linewidth=2)
for n in range(1, 7):
    w_approx = solutions[n][3]
    w_approx_func = lambdify(x, w_approx.subs(y, 4).subs(a, 8).subs(b, 8).subs(solutions[n][2]), 'numpy')
    w_approx_vals = w_approx_func(x_vals)
    plt.plot(x_vals, w_approx_vals, label=f'Ritz Order {n} (Degree {min(4+n, 6)})')
plt.xlabel('x (m)')
plt.ylabel('Displacement w(x, b/2) (m)')
plt.title('Kirchhoff Plate: Polynomial Approximations vs Analytical')
plt.legend()
plt.grid(True)
plt.savefig('kirchhoff_plate_displacement.png')
plt.close()

# Plot potential energy convergence
plt.figure(figsize=(10, 6))
plt.plot(range(1, 7), potential_energies, 'bo-', label='Total Potential Energy')
plt.xlabel('Ritz Order (Number of Terms)')
plt.ylabel('Total Potential Energy (J)')
plt.title('Convergence of Total Potential Energy')
plt.legend()
plt.grid(True)
plt.savefig('kirchhoff_plate_energy_convergence.png')
plt.close()

# Print potential energies
print("Total Potential Energy for Each Order:")
for n in range(1, 7):
    print(f"Order {n} (Degree {min(4+n, 6)}): {potential_energies[n-1]:.2e} J")

Total Potential Energy for Each Order:
Order 1 (Degree 5): -6.39e+02 J
Order 2 (Degree 6): -6.94e+02 J
Order 3 (Degree 6): -7.34e+02 J
Order 4 (Degree 6): -7.36e+02 J
Order 5 (Degree 6): -7.36e+02 J
Order 6 (Degree 6): -7.37e+02 J


In [41]:
# Numerical evaluation along y = b/2
x_vals = np.linspace(0, 8, 100)
y_val = 4
w_analytical_vals = w_analytical_func(x_vals, y_val)


In [53]:
# Plot displacement
plt.figure(figsize=(10, 6))
plt.plot(x_vals, w_analytical_vals, 'k-', label='Analytical Solution', linewidth=2)
for n in range(1, 7):
    w_approx = solutions[n][3]
    w_approx_func = lambdify(x, w_approx.subs(y, 4).subs(a, 8).subs(b, 8).subs(solutions[n][2]), 'numpy')
    w_approx_vals = w_approx_func(x_vals)
    plt.plot(x_vals, w_approx_vals, label=f'Ritz Order {n} (Degree {min(4+n, 6)})')
plt.xlabel('x (m)')
plt.ylabel('Displacement w(x, b/2) (m)')
plt.title('Kirchhoff Plate: Polynomial Approximations vs Analytical')
plt.legend()
plt.grid(True)
plt.savefig('kirchhoff_plate_displacement.png')
plt.close()

# Plot potential energy convergence
plt.figure(figsize=(10, 6))
plt.plot(range(1, 7), potential_energies, 'bo-', label='Total Potential Energy')
plt.xlabel('Ritz Order (Number of Terms)')
plt.ylabel('Total Potential Energy (J)')
plt.title('Convergence of Total Potential Energy')
plt.legend()
plt.grid(True)
plt.savefig('kirchhoff_plate_energy_convergence.png')
plt.close()

# Print potential energies
print("Total Potential Energy for Each Order:")
for n in range(1, 7):
    print(f"Order {n} (Degree {min(4+n, 6)}): {potential_energies[n-1]:.2e} J")

Total Potential Energy for Each Order:
Order 1 (Degree 5): -6.39e+02 J
Order 2 (Degree 6): -6.94e+02 J
Order 3 (Degree 6): -7.34e+02 J
Order 4 (Degree 6): -7.36e+02 J
Order 5 (Degree 6): -7.36e+02 J
Order 6 (Degree 6): -7.37e+02 J


In [55]:
# Compute for Order 6
n = 6
K, F = compute_system(n)
c = Matrix(symbols(f'c1:{n+1}'))
solution = solve(K * c - F, c)
w_approx = sum(solution.get(symbols(f'c{i+1}'), 0) * phi[i] for i in range(n))

In [58]:
# Compute M_x
M_x = -D * (w_approx.diff(x, 2) + nu * w_approx.diff(y, 2))

pprint(M_x)

54.0033581920217⋅x⋅(x - 8) - 93.7873447592549⋅x⋅(y - 8) - 802.81967113921⋅x -  ↪

↪ 28.1362034277766⋅y⋅(x - 8) + 180.011193973406⋅y⋅(y - 8) - 2676.06557046404⋅y ↪

↪  - 60.9617740935157⋅(x - 8)⋅(y - 8) + 27831.081932826


In [59]:
# Numerical evaluation
x_vals = np.linspace(0, 8, 50)
y_vals = np.linspace(0, 8, 50)
X, Y = np.meshgrid(x_vals, y_vals)
w_func = lambdify((x, y), w_approx.subs(a, 8).subs(b, 8).subs(solution), 'numpy')
M_x_func = lambdify((x, y), M_x.subs(a, 8).subs(b, 8).subs(solution), 'numpy')
w_vals = w_func(X, Y)
M_x_vals = M_x_func(X, Y)

In [60]:
# Plot w(x, y)
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, w_vals, cmap='viridis')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_zlabel('Displacement w(x, y) (m)')
ax.set_title('Displacement w(x, y) for Order 6 Polynomial Approximation')
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
plt.savefig('kirchhoff_plate_displacement_3d.png')
plt.close()

In [61]:
# Plot M_x(x, y)
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, M_x_vals, cmap='plasma')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_zlabel('Moment M_x(x, y) (N·m/m)')
ax.set_title('Moment M_x(x, y) for Order 6 Polynomial Approximation')
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
plt.savefig('kirchhoff_plate_moment_mx_3d.png')
plt.close()

In [62]:
# Analytical solution for reference (Navier’s method)
m, n = symbols('m n', integer=True)
w_mn = (16 * q0 / (pi**2 * m * n)) / (D * pi**4 * ((m**2 / a**2) + (n**2 / b**2))**2) * sin(m * pi * x / a) * sin(n * pi * y / b)
w_analytical = 0
m_max, n_max = 5, 5
for i in range(1, m_max + 1, 2):
    for j in range(1, n_max + 1, 2):
        w_analytical += w_mn.subs({m: i, n: j})
w_analytical_func = lambdify((x, y), w_analytical.subs({a: 8, b: 8, D: D, q0: 5e3}), 'numpy')
w_analytical_vals = w_analytical_func(X, Y)

# Plot analytical w(x, y)
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')
surf = ax.plot_surface(X, Y, w_analytical_vals, cmap='viridis')
ax.set_xlabel('x (m)')
ax.set_ylabel('y (m)')
ax.set_zlabel('Displacement w(x, y) (m)')
ax.set_title('Analytical Displacement w(x, y)')
fig.colorbar(surf, ax=ax, shrink=0.5, aspect=5)
plt.savefig('kirchhoff_plate_analytical_displacement_3d.png')
plt.close()